In [1]:
import os
import pandas as pd
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots
import numpy
import scipy.optimize as optimize

# colorbar
colorscale=[
        [0, "rgb(183,0,0)"],
        [0.1, "rgb(183,0,0)"],
        [0.1, "rgb(255,56,0)"],
        [0.2, "rgb(255,56,0)"],
        [0.2, "rgb(255,90,0)"],
        [0.3, "rgb(255,90,0)"],
        [0.3, "rgb(255,146,0)"],
        [0.4, "rgb(255,146,0)"],
        [0.4, "rgb(255,203,0)"],
        [0.5, "rgb(255,203,0)"],
        [0.5, "rgb(195,255,0)"],
        [0.6, "rgb(195,255,0)"],
        [0.6, "rgb(153,204,51)"],
        [0.7, "rgb(153,204,51)"],
        [0.7, "rgb(67,166,12)"],
        [0.8, "rgb(67,166,12)"],
        [0.8, "rgb(51,153,0)"],
        [0.9, "rgb(51,153,0)"],
        [0.9, "rgb(38,98,54)"],
        [1, "rgb(38,98,54)"],]
colorbar=dict(tickmode='array',tickvals=[0.05, 0.15,0.25,0.35,0.45,0.55,0.65,0.75,0.85,0.95],ticktext=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-90','90-100'],len=1.05,ticklabeloverflow='allow')

filename='vdi4657_chapter_9-2-3-2_results.csv'

In [2]:
# ONLY NECCECARY DIRECTLY AFTER THE PARAMETER-VARIATION
# read raw results and save

#df=pd.DataFrame()
#for folder in os.listdir('results'):
#    try:
#        df=pd.concat([df,pd.read_csv(r'results/'+folder+'/results_kpi.csv')])
#    except:
#        print('No `results_kpi.csv` in ' + folder)
#df.sort_values('e_bat')
#df.sort_values('p_chp')
#df.to_csv(filename)

In [3]:
#Fit-Functions
guess = (1.0,1.0,1.0,1.0)
def func_a(data, k1,k2,k3,k4):
    return numpy.maximum((k1*data[0]+k2)*(k4*data[1]+1)/((data[0]+k3)*(data[1]+1)),0)
def func_e(data, k1,k2,k3,k4):
    return numpy.minimum((k1*data[0]+k2)*(k4*data[1]+1)/((data[0]+k3)*(data[1]+1)),1)

In [ ]:
# TODO Auswertung erfolgt bisher nur über ein Profil
# TODO Schleife aufbauen und Fit-Parameter inkl. RMSE als CSV abspeichern
# TODO Grafiken final hübsch machen und als SVG, PNG, PDF speichern. 
# Titel: "Lastprofil-Name"_[pv,chp]_[region]_[type]_[year]_[a,e]_[sim,fit]

In [8]:
# Profil
profile = 'LP_W_EFH'

# Autarkiegrad
# load results and fit
df=pd.read_csv(filename)
# Filter data for fit
df=df.loc[df['electric_loadprofile']==profile,:]
df=df.loc[df['p_chp']>0,:]
xData = df['p_chp'].values
yData = df['e_bat'].values
zData = df['autarkiegrad'].values
# place the data in a single list
data = [xData, yData, zData]
params_a, pcov = optimize.curve_fit(func_a, [xData,yData], zData, guess)

# calc values with fitting functions
df1=pd.DataFrame()
CHP = list(numpy.arange(0.05,0.55,0.05))
BAT = list(numpy.arange(0,1.1,0.1))
e_bat=[]
p_chp=[]
eigen=[]
aut=[]
for bat in BAT:
    for chp in CHP:
        e_bat.append(bat)
        p_chp.append(chp)
        aut.append(func_a([chp,bat],params_a[0],params_a[1],params_a[2],params_a[3]))
df1['e_bat']=e_bat
df1['p_chp']=p_chp
df1['autarkiegrad']=aut
fig= make_subplots(1,2)
# Filter for plot
df=df.loc[df['p_chp']>=0.05,:]
df=df.loc[df['p_chp']!=0.075,:]
df=df.loc[df['e_bat']>=0.10,:]
df=df.loc[df['e_bat']!=0.15,:]
fig.add_trace(go.Heatmap(x=df['p_chp'].values,
                                y=df['e_bat'].values,
                                z=df['autarkiegrad'].values,
                                text=df['autarkiegrad'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis'),1,1)

fig.add_trace(go.Heatmap(x=df1['p_chp'].values,
                                y=df1['e_bat'].values,
                                z=df1['autarkiegrad'].values,
                                text=df1['autarkiegrad'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis',
                                opacity=1,),1,2)
fig.update_layout(title='Autarkiegrad simuliert / gefittet',width=900.5,height=500,coloraxis = {'cmin':0,'cmax':1.0,'colorscale':colorscale,'colorbar':colorbar,'autocolorscale':False},)
fig.update_yaxes(tick0=0,dtick=0.1,tickangle=270,title='C_eff_sp in kWh/MWh')
fig.update_xaxes(tick0=0,dtick=0.05,title='p_chp_sp in kWp/MWh')
fig.show()

print('fitted parameters', params_a)
modelPredictions = func_a(data, *params_a) 
absError = modelPredictions - zData
SE = numpy.square(absError) # squared errors
MSE = numpy.mean(SE) # mean squared errors
RMSE = numpy.sqrt(MSE) # Root Mean Squared Error, RMSE
Rsquared = 1.0 - (numpy.var(absError) / numpy.var(zData))
print('RMSE:', RMSE)
print('R-squared:', Rsquared)

fitted parameters [ 0.71417769 -0.00968332  0.06312682  1.70062092]
RMSE: 0.02569247633283476
R-squared: 0.9835488765416549


In [9]:
# Eigenverbrauchsanteil
# load results and fit
df=pd.read_csv(filename)
# Filter data for fit
df=df.loc[df['electric_loadprofile']==profile,:]
df=df.loc[df['p_chp']>0,:]
df=df.loc[df['eigenverbrauch']<1,:]
xData = df['p_chp'].values
yData = df['e_bat'].values
zData = df['eigenverbrauch'].values
# place the data in a single list
data = [xData, yData, zData]
params_e, pcov = optimize.curve_fit(func_e, [xData,yData], zData, guess)

# calc values with fitting functions
df1=pd.DataFrame()
CHP = list(numpy.arange(0.05,0.55,0.05))
BAT = list(numpy.arange(0,1.1,0.1))
e_bat=[]
p_chp=[]
eigen=[]
for bat in BAT:
    for chp in CHP:
        e_bat.append(bat)
        p_chp.append(chp)
        eigen.append(func_e([chp,bat],params_e[0],params_e[1],params_e[2],params_e[3]))
df1['e_bat']=e_bat
df1['p_chp']=p_chp
df1['eigenverbrauch']=eigen
fig= make_subplots(1,2)
# Filter for plot
df=df.loc[df['p_chp']>=0.05,:]
df=df.loc[df['p_chp']!=0.075,:]
df=df.loc[df['e_bat']>=0.10,:]
df=df.loc[df['e_bat']!=0.15,:]
fig.add_trace(go.Heatmap(x=df['p_chp'].values,
                                y=df['e_bat'].values,
                                z=df['eigenverbrauch'].values,
                                text=df['eigenverbrauch'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis'),1,1)

fig.add_trace(go.Heatmap(x=df1['p_chp'].values,
                                y=df1['e_bat'].values,
                                z=df1['eigenverbrauch'].values,
                                text=df1['eigenverbrauch'].round(2),
                                texttemplate="%{text}",
                                textfont={"size":10},
                                coloraxis='coloraxis',
                                opacity=1,),1,2)
fig.update_layout(title='Eigenverbrauchsanteil simuliert / gefittet',width=900.5,height=500,coloraxis = {'cmin':0,'cmax':1.0,'colorscale':colorscale,'colorbar':colorbar,'autocolorscale':False},)
fig.update_yaxes(tick0=0,dtick=0.1,tickangle=270,title='C_eff_sp in kWh/MWh')
fig.update_xaxes(tick0=0,dtick=0.05,title='p_chp_sp in kWp/MWh')
fig.show()

print('fitted parameters', params_e)
modelPredictions = func_e(data, *params_e) 
absError = modelPredictions - zData
SE = numpy.square(absError) # squared errors
MSE = numpy.mean(SE) # mean squared errors
RMSE = numpy.sqrt(MSE) # Root Mean Squared Error, RMSE
Rsquared = 1.0 - (numpy.var(absError) / numpy.var(zData))
print('RMSE:', RMSE)
print('R-squared:', Rsquared)

fitted parameters [-0.01475371  0.14505153  0.07748017  1.915281  ]
RMSE: 0.014704807868488714
R-squared: 0.997212400249596
